# EEEN30131 Power Flow Analysis & Power System Control Coursework

***&copy; 2024 Martínez Ceseña — University of Manchester, UK***

This notebook provides the tools required to complete EEEN30131's coursework, and should be used alongside the **EEEN30131 Coursework Instructions.docx** file.

## List of contents

- [Important dates](#Important-dates)
- [Before we begin](#Before-we-begin)
- [Practise - Getting familiar with Jupyter notebooks](#Practise---Getting-familiar-with-Jupyter-notebooks)
  - [Activating the notebook](#Activating-the-notebook)
  - [Using the tools presented in this notebook](#Using-the-tools-presented-in-this-notebook)
- [Part A – Power Flow Analysis](#Part-A-–-Power-Flow-Analysis)
  - [Task A.1 - Load Variations](#Task-A.1---Load-Variations)
    - [A.1.1 (Record in excel template) [10 marks]](#A.1.1-(Record-in-excel-template)-[10-marks])
    - [A.1.2 (Record in excel template) [10 marks]](#A.1.2-(Record-in-excel-template)-[10-marks])
    - [A.1.3 (Report) [10 marks]](#A.1.3-(Report)-[10-marks])
  - [Task A.2 Generation Variations](#Task-A.2-Generation-Variations)
    - [A.2.1 (Report) [7 marks]](#A.2.1-(Report)-[7-marks])
    - [A.2.2 (Report) [3 marks]](#A.2.2-(Report)-[3-marks])
    - [A.2.3 (Report) [10 marks]](#A.2.3-(Report)-[10-marks])
- [Part B - Power System Control](#Part-B---Power-System-Control)
  - [Task B.1 - Primary frequency control for systems with low inertia](#Task-B.1---Primary-frequency-control-for-systems-with-low-inertia)
    - [B.1.1 (Record in excel template) [10 marks]](#B.1.1-(Record-in-excel-template)-[10-marks])
    - [B.1.2 (Report) [7 marks]](#B.1.2-(Report)-[7-marks])
  - [Task B.2 - Primary frequency control – Contributions of loads](#Task-B.2---Primary-frequency-control-–-Contributions-of-loads)
    - [B.2.1 (Record in excel template) [10 marks]](#B.2.1-(Record-in-excel-template)-[10-marks])
    - [B.2.2 (Report) [5 marks]](#B.2.2-(Report)-[5-marks])
  - [Task B.3 - Voltage control](#Task-B.3---Voltage-control)
    - [B.3.1 (Record in excel template) [10 marks]](#B.3.1-(Record-in-excel-template)-[10-marks])
    - [B.3.2 (Report) [5 marks]](#B.3.2-(Report)-[5-marks])
- [Summary](#Summary)

[Back to top](#EEEN30131-Power-Flow-Analysis-&-Power-System-Control-Coursework)

## Important dates

Review the EEEN30131 [Blackboard space](https://online.manchester.ac.uk/webapps/blackboard/content/listContentEditable.jsp?content_id=_15046413_1&course_id=_78315_1) and look for the following:
* Deadline to upload pre-lab exercise.
* The date when and place where tab takes place.
* Deadline to submit the coursework (i.e., report and excel template).

## Before we begin

Before we begin, remember that you should have completed the pre-lab exercise and filled the **EEEN30131_PSA_PreWork_Template.xlsm** template. Otherwise, you will **lose 3 marks** and will not have the information required to complete the exercises presented below.

If you have completed the pre-lab exercise, you can proceed.
- This notebook will provide all the tools required to address every exercise presented in the laboratory.
- The tools have been discussed in the lectures and presented in relevant jupyter notebooks. Please refer to the lectures if you need more details about the tools.
- To import the tools, we will use a python library (*nbimporter*) that allows us to import tools developed in other jupyter notebooks.

If this data notebook is being used in Jupyter lite, the folders where the python code that supports this notebook are stored, have to be enabled.

In [ ]:
from pathlib import Path
if Path.cwd().drive == '':
    a_dir = Path("dir")
    a_dir.mkdir(exist_ok=True)

The following conventional python libraries are used:

In [ ]:
import numpy
try:
    import ipywidgets as widgets
except:
    import micropip
    await micropip.install('ipywidgets')
    import ipywidgets as widgets
from ipywidgets import interact
import cmath
import numpy
import matplotlib.pyplot as plt
import math
import warnings
warnings.filterwarnings("ignore")

The following tools that were developed for EEEN30131 are loaded:

In [1]:
from Code.Wk2_EEEN30131 import get_Bus_Type, develop_PF_Equations
from Code.Wk3_EEEN30131 import Newtons_Method, Visualize_Elec
from Code.Wk4_EEEN30131 import get_Primary_Response, get_pu, get_Δw, get_step
from Code.Wk5_EEEN30131 import get_Tie_Data, get_TieLine, interconnected_frequency_primary
from Code.Wk5_EEEN30131 import interconnected_power, plot_Δw, plot_Ppu, print_TieLine
from Code.Wk6_EEEN30131 import get_Ybus, get_Parameters

FInally, to facilitate the work in this lab, a bespoke display method is introduced to produce the results in the units and format required for your coursework.

In [3]:
def Visualize_Lab(Connectivity, Load, P, Q, Base, V_All, 𝜃_All, Succes):
    if Succes:
        Results = get_Parameters(Connectivity, V_All, 𝜃_All, Succes)

        print('LOADS:')
        for xb in range(len(V_All)):
            print('P%d = %10.4f [GW], Q%d = %10.4f [GVAr]'%(xb+1, P[xb]*Base, xb+1, Q[xb]*Base))

        print('\nGENERATORS:')
        for xg in range(len(V_All)):
            G = Results['Net_Power'][xg]/1000 + Load[xg][1]*Base
            if abs(G) > 0.001:
                print('G%d = %7.4f + j %7.4f [GVA]'%(xg+1, G.real, G.imag))

        print('\nVOLTAGES:')
        for xb in range(len(V_All)):
            print('V%d =  %7.4f ∠ %8.4f [pu][deg]'%(xb+1, V_All[xb], 𝜃_All[xb]*180/math.pi))
        
        NoBranch = len(Results['Connectivity'])
        print('\nPOWER FLOWS:')
        for xb in range(NoBranch):
            print('%2.0f - %2.0f %7.4f + j %7.4f [GVA] to %7.4f + j %7.4f [GVA]'%
                  (Results['Connectivity'][xb][0],Results['Connectivity'][xb][1],
                  Results['Sending_Power'][xb].real/1000, Results['Sending_Power'][xb].imag/1000,
                  Results['Receiving_Power'][xb].real/1000, Results['Receiving_Power'][xb].imag/1000))

        print('\nLOSSES:')
        for xb in range(NoBranch):
            print('%2.0f - %2.0f %7.4f + j %7.4f [GVA]'%
                  (Results['Connectivity'][xb][0],Results['Connectivity'][xb][1],
                  Results['Loss'][xb].real/1000, Results['Loss'][xb].imag/1000))
        return Results
    return False

[Back to top](#EEEN30131-Power-Flow-Analysis-&-Power-System-Control-Coursework)

## Practise - Getting familiar with Jupyter notebooks

### Activating the notebook

Remember that this notebook is displaying the results that were generated the last time the file was saved. However, the notebook may not be operational. 

To activate the notebook, you can restart and run all cells in the notebook by selecting the "Restart & Run All" option within the "Kernel menu", which is located on the top left of this browser.

`Kernel/Restart & Run All`

Once you have activated the notebook, it is possible to use the different tools available. 

Let us check if the notebook is active by running the cell below. For this purpose, click on the cell and, afterwards, click on ▶.

In [4]:
t = numpy.linspace(0,5,6)
print('The notebook is active and can use different tools')
print('such as the numpy library which allows us to create')
print('vectors such as:')
print(t)

The notebook is active and can use different tools
such as the numpy library which allows us to create
vectors such as:
[0. 1. 2. 3. 4. 5.]


[Back to top](#EEEN30131-Power-Flow-Analysis-&-Power-System-Control-Coursework)

### Using the tools presented in this notebook

The tools in this take inputs from graphical user interfaces. To be more specific, you do not need to edit the code, you just need to manipulate the sliders.

For this purpose, play with the tool presented below:
- Drag the slider and check how the display changes:
  - Can you select a number that is outside the 0 - 10 range?
  - Can you select a non-integer number (e.g., 7.5)?
- Click on the number on the right of the slider and type a number:
  - Does the number change if you do not press `Enter`?
  - Can you select a number that is outside the 0 - 10 range?
  - Can you select a non-integer number (e.g., 7.5)?

In [5]:
@interact
def A_1_1(Input = widgets.FloatSlider(min=0, max=10,step=1,value=5, description='Input',
                                    continuous_update=False, readout_format='.4f')):
    print('The value of the input is %.4f'%Input)

interactive(children=(FloatSlider(value=5.0, continuous_update=False, description='Input', max=10.0, readout_f…

Make sure that you understand the outputs provided by the interactive cells, as you will need to record some of the information and use it to inform your discussion. If something is unclear, please ask the demonstrators (during the laboratory) or post your questions in the discussion boards.

[Back to top](#EEEN30131-Power-Flow-Analysis-&-Power-System-Control-Coursework)

## Part A – Power Flow Analysis

In this part of the laboratory, you will be simulating steady-state power flows for the simplified GB system presented in Figure 1.

![Week06_3Bus_UK.png](Figures/Week06_3Bus_UK.png)
<div align="center">
Figure 1: Simplified equivalent model of the GB system.
</div>

To complete this part of the laboratory, you will have to complete two tasks:
- Task A.1 involves performing a series of power flows for the GB system using predefined load values (defined in pre-lab exercise) to simulate and discuss the conditions of the system. This task will allow you to explore how the conditions of the power flow are affected by the ever changing demands.
- Task A.2 involves changing the set-points of some generators in the simplified GB system until reaching its limits. This task will allow you to explore how non-synchronous generators (renewable generators), such as wind generators, can cause network stress.

[Back to top](#EEEN30131-Power-Flow-Analysis-&-Power-System-Control-Coursework)

### Task A.1 - Load Variations

This task involves three activities, each one with a value of 10 marks:
- A.1.1: Use the tool presented to simulate power flows under different conditions and a load power factor of 0.95 lagging. The results should be recorded in the excel template provided.
- A.1.2: Use the tool presented to simulate power flows under different conditions and a load power factor of 0.92 lagging. The results should be recorded in the excel template provided.
- A.1.3: Analyse your results and discuss your findings in the report.

[Back to top](#EEEN30131-Power-Flow-Analysis-&-Power-System-Control-Coursework)

#### A.1.1 (Record in excel template) [10 marks]

To complete this activity you will have to:
- Perform power flow simulations for the simplified GB systems.
  - Consider a load power factor of **0.95 lagging**.
  - Simulate the system as the load varies from 40% to 100% of peak load (in steps of 10%); you should have these load values in your **EEEN30131_PSA_PreWork_Template.xlsm**.
- Set simulation rules for the generators:
  - Bus 1 (Scotland) is modelled as a PV bus:
    - The voltage set-point of the synchronous generator in Scotland is set to 1.02 pu.
    - The synchronous generator in Scotland is dispatched with an active power set-point that is 1.0 GW higher than the active power load demand in Scotland. So if the load in Scotland is 2.4 GW, then the active power set-point in Scotland should be set to 3.4 GW.
  - Bus 2 (Northern England) is modelled as a PQ bus.
  - Bus 3 (England and Wales) is modelled as a Slack bus:
    - The voltage magnitude of the synchronous generator in England is set to 1.0 pu.
    - The voltage angle of the synchronous generator in England is set to 0 deg.
- Record your results in the **EEEN30131_PSA_Coursework_Template.xlsm**
  - Select the A.1.1 tab.
  - Record your results to four decimal places.

The tool below can be used to perform the power flow studies. The tool already includes the GB network information, and the bus types and voltage settings. Accordingly, you will have to provide the following inputs:
- Ld P1 (GW): Active load in Bus 1 in GW.
- Ld Q1 (GVAr): Reactive load in Bus 1 in GVAr.
- Ld P2 (GW): Active load in Bus 2 in GW.
- Ld Q2 (GVAr): Reactive load in Bus 2 in GVAr.
- Ld P3 (GW): Active load in Bus 3 in GW.
- Ld Q3 (GVAr): Reactive load in Bus 3 in GVAr.
- G P1 (GW): Active power output set point GW for the synchronous generator in Scotland.

In [6]:
@interact
def A_1_1(LP1 = widgets.FloatSlider(min=1,max=5,step=0.5,value=2, description='Ld P1 (GW)',
                                    continuous_update=False, readout_format='.4f'),
          LQ1 = widgets.FloatSlider(min=0.49305795,max=3,step=0.16434205,value=0.6574, description='Ld Q1 (GVAr)',
                                    continuous_update=False, readout_format='.4f'),
          LP2 = widgets.FloatSlider(min=0.8,max=5,step=0.4,value=1.6, description='Ld P2 (GW)',
                                    continuous_update=False, readout_format='.4f'),
          LQ2 = widgets.FloatSlider(min=0.39442636,max=2,step=0.13147364,value=0.5259, description='Ld Q2 (GVAr)',
                                    continuous_update=False, readout_format='.4f'),
          LP3 = widgets.FloatSlider(min=4,max=40,step=4,value=16, description='Ld P3 (GW)',
                                    continuous_update=False, readout_format='.4f'),
          LQ3 = widgets.FloatSlider(min=3.94416358,max=20,step=1.31473642,value=5.2589, description='Ld Q3 (GVAr)',
                                    continuous_update=False, readout_format='.4f'),
          GP1 = widgets.FloatSlider(min=1,max=6,step=0.5,value=3, description='G P1 (GW)',
                                    continuous_update=False, readout_format='.4f')):

    # Get active and reactive loads
    Base = 1/10
    P = [LP1/Base, LP2/Base, LP3/Base]
    Q = [LQ1/Base, LQ2/Base, LQ3/Base]

    Connectivity = [
        [1, 2, complex(0.0003, 0.0030), 1.2],
        [2, 3, complex(0.0015, 0.0150), 2.2]
    ]
    Load = [
        [1, complex(P[0], Q[0])],
        [2, complex(P[1], Q[1])],
        [3, complex(P[2], Q[2])]
    ]
    Generator = [
        {'Bus':3, 'V':1, '𝜃':0 },
        {'Bus':1, 'P':GP1/Base, 'V':1.02 }
    ]
    
    Flg = True
    for xBus in range(3):
        if abs(P[xBus]/(P[xBus]**2+Q[xBus]**2)**0.5-0.95) > 0.01:
            Flg = False
    Per = P[0]/5
    if abs(P[1]/4-Per) > 0.1 or abs(P[2]/40-Per) > 0.1:
        Flg = False
    if abs(GP1-LP1-1) > 0.01:
        Flg = False
    if Flg:
        print('RESULTS FOR A POWER FACTOR OF 0.95 LAGGING:')
    else:
        print('WARNING THE SETTINGS OF THIS STUDY DO NOT SEEM CORRECT')
    print()
    
    Ybus = get_Ybus(Connectivity, True, False)

    P_Data, Q_Data = develop_PF_Equations(Load, Generator, Ybus, True, False)
    Bus_Data, Bus_Type = get_Bus_Type(Ybus, Load, Generator)

    V_All, 𝜃_All, Threshold, Succes = Newtons_Method(P_Data, Q_Data, Bus_Data, Bus_Type, Generator, 0)
    Visualize_Lab(Connectivity, Load, P, Q, Base, V_All, 𝜃_All, Succes)

interactive(children=(FloatSlider(value=2.0, continuous_update=False, description='Ld P1 (GW)', max=5.0, min=1…

[Back to top](#EEEN30131-Power-Flow-Analysis-&-Power-System-Control-Coursework)

#### A.1.2 (Record in excel template) [10 marks]

To complete this activity you will have to perform a similar set of studies as those performed above in Task A.1.1, but now using a load power factor of **0.92 lagging**. 

The full set of instructions for this task are provided below:
- Perform power flow simulations for the simplified GB systems.
  - Consider a load power factor of **0.92 lagging**.
  - Simulate the system as the load varies from 40% to 100% of peak load (in steps of 10%); you should have these load values in your **EEEN30131_PSA_PreWork_Template.xlsm**.
- Set simulation rules for the generators:
  - Bus 1 (Scotland) is modelled as a PV bus:
    - The voltage set-point of the synchronous generator in Scotland is set to 1.02 pu.
    - The synchronous generator in Scotland is dispatched with an active power set-point that is 1.0 GW higher than the active power load demand in Scotland. So if the load in Scotland is 2.4 GW, then the active power set-point in Scotland should be set to 3.4 GW. 
  - Bus 2 (Northern England) is modelled as a PQ bus.
  - Bus 3 (England and Wales) is modelled as a Slack bus:
    - The voltage magnitude of the synchronous generator in England is set to 1.0 pu.
    - The voltage angle of the synchronous generator in England is set to 0 deg.
- Record your results in the **EEEN30131_PSA_Coursework_Template.xlsm**.
  - Select the A.1.1 tab.
  - Record your results to four decimal places.

The tool below can be used to perform the power flow studies. The tool already includes the GB network information, and the bus types and voltage settings. Accordingly, you will have to provide the following inputs:
- Ld P1 (GW): Active load in Bus 1 in GW.
- Ld Q1 (GVAr): Reactive load in Bus 1 in GVAr.
- Ld P2 (GW): Active load in Bus 2 in GW.
- Ld Q2 (GVAr): Reactive load in Bus 2 in GVAr.
- Ld P3 (GW): Active load in Bus 3 in GW.
- Ld Q3 (GVAr): Reactive load in Bus 3 in GVAr.
- G P1 (GW): Active power output set point GW for the synchronous generator in Scotland.

In [7]:
@interact
def A_1_1(LP1 = widgets.FloatSlider(min=1,max=5,step=0.5,value=2, description='Ld P1 (GW)',
                                    continuous_update=False, readout_format='.4f'),
          LQ1 = widgets.FloatSlider(min=0.4260018,max=3,step=0.21299910,value=0.8520, description='Ld Q1 (GVAr)',
                                    continuous_update=False, readout_format='.4f'),
          LP2 = widgets.FloatSlider(min=0.8,max=5,step=0.4,value=1.6, description='Ld P2 (GW)',
                                    continuous_update=False, readout_format='.4f'),
          LQ2 = widgets.FloatSlider(min=0.34080144,max=2,step=0.17039928,value=0.6816, description='Ld Q2 (GVAr)',
                                    continuous_update=False, readout_format='.4f'),
          LP3 = widgets.FloatSlider(min=4,max=40,step=4,value=16, description='Ld P3 (GW)',
                                    continuous_update=False, readout_format='.4f'),
          LQ3 = widgets.FloatSlider(min=3.40801428,max=20,step=1.70399286,value=6.8160, description='Ld Q3 (GVAr)',
                                    continuous_update=False, readout_format='.4f'),
          GP1 = widgets.FloatSlider(min=1,max=6,step=0.5,value=3, description='G P1 (GW)',
                                    continuous_update=False, readout_format='.4f')):

    # Get active and reactive loads
    Base = 1/10
    P = [LP1/Base, LP2/Base, LP3/Base]
    Q = [LQ1/Base, LQ2/Base, LQ3/Base]

    Connectivity = [
        [1, 2, complex(0.0003, 0.0030), 1.2],
        [2, 3, complex(0.0015, 0.0150), 2.2]
    ]
    Load = [
        [1, complex(P[0], Q[0])],
        [2, complex(P[1], Q[1])],
        [3, complex(P[2], Q[2])]
    ]
    Generator = [
        {'Bus':3, 'V':1, '𝜃':0 },
        {'Bus':1, 'P':GP1/Base, 'V':1.02 }
    ]
    
    Flg = True
    for xBus in range(3):
        if abs(P[xBus]/(P[xBus]**2+Q[xBus]**2)**0.5-0.92) > 0.01:
            Flg = False
    Per = P[0]/5
    if abs(P[1]/4-Per) > 0.1 or abs(P[2]/40-Per) > 0.1:
        Flg = False
    if abs(GP1-LP1-1) > 0.01:
        Flg = False
    if Flg:
        print('RESULTS FOR A POWER FACTOR OF 0.92 LAGGING:')
    else:
        print('WARNING THE SETTINGS OF THIS STUDY DO NOT SEEM CORRECT')
    print()

    Ybus = get_Ybus(Connectivity, True, False)

    P_Data, Q_Data = develop_PF_Equations(Load, Generator, Ybus, True, False)
    Bus_Data, Bus_Type = get_Bus_Type(Ybus, Load, Generator)

    V_All, 𝜃_All, Threshold, Succes = Newtons_Method(P_Data, Q_Data, Bus_Data, Bus_Type, Generator, 0)
    Visualize_Lab(Connectivity, Load, P, Q, Base, V_All, 𝜃_All, Succes)

interactive(children=(FloatSlider(value=2.0, continuous_update=False, description='Ld P1 (GW)', max=5.0, min=1…

[Back to top](#EEEN30131-Power-Flow-Analysis-&-Power-System-Control-Coursework)

#### A.1.3 (Report) [10 marks]

Discuss in your report the results you have obtained.

You might want to include some of the following in your discussion:
- Descriptions and explanations about how the results change with varying loading.
- Comparisons about the results with different load power factors.
- Data visualisation that supports your discussion. 
- Information from the lectures or literature that supports your discussion.

[Back to top](#EEEN30131-Power-Flow-Analysis-&-Power-System-Control-Coursework)

### Task A.2 Generation Variations

This task involves three activities worth seven, three, and ten marks, respectively:
- A.2.1: Use the tool presented to simulate power flows as the non-synchronous generation in Scotland increases. Produce a table with the voltage magnitudes and angles and add it to your report.
- A.2.2: Building on the simulations in A.2.1, determine the maximum output of the generator in Scotland (to the nearest 100 MW) which can be accommodated by the system without (i.e., without exceeding its statutory limits). State this value in your report.
- A.2.3: Analyse your results and discuss your findings in the report.

[Back to top](#EEEN30131-Power-Flow-Analysis-&-Power-System-Control-Coursework)

#### A.2.1 (Report) [7 marks]

To complete this activity you will have to:
- Perform power flow simulations for the simplified GB systems.
  - Consider 100% loading conditions (peak demand) and a power factor of **0.95 lagging**.
  - Assume there is a synchronous generator and a non-synchronous generator in Scotland.
- Set simulation rules for the generators:
  - Bus 1 (Scotland) is modelled as a PV bus with both a synchronous and a non-synchronous (Wind) generator:
    - The voltage set-point of the synchronous generator in Scotland is set to 1.02 pu.
    - The active power output of the synchronous generator is set to 7 GW.
    - The active power output of the wind generator is initially set to zero, but is increased until the statutory limits of the network are exceeded (i.e., voltages must be kept between 0.95 pu and 1.05 pu). 
  - Bus 2 (Northern England) is modelled as a PQ bus.
  - Bus 3 (England and Wales) is modelled as a Slack bus:
    - The voltage magnitude of the synchronous generator in England is set to 1.0 pu.
    - The voltage angle of the synchronous generator in England is set to 0 deg.
- Create a table to record the results in your report (do not add these results to the excel template).
  - Record the voltage magnitudes and angles for every bus in the table.
  - Record your results to four decimal places.
  - You will have to include enough values in the table to illustrate how the increased wind generation in Scotland impacts the voltages across the network, e.g., are voltages increasing or decreasing?

The tool below can be used to perform the power flow studies. The tool already includes the GB network information, and the bus types and voltage settings. 

You will first need to set the loads in every bus as well as the output of the synchronous generator in Scotland:
- Ld P1 (GW): Active load in Bus 1 in GW
- Ld Q1 (GVAr): Reactive load in Bus 1 in GVAr
- Ld P2 (GW): Active load in Bus 2 in GW
- Ld Q2 (GVAr): Reactive load in Bus 2 in GVAr
- Ld P3 (GW): Active load in Bus 3 in GW
- Ld Q3 (GVAr): Reactive load in Bus 3 in GVAr
- G P1 (GW): Active power output set point in GW for the synchronous generator in Scotland

Once you have set the values above, you can change the output of the wind generator by adjusting the following input:
- Wind (GW): Active power output set point in GW for the non-synchronous generator in Scotland

In [8]:
@interact
def A_1_1(LP1 = widgets.FloatSlider(min=1,max=5,step=0.5,value=2, description='Ld P1 (GW)',
                                    continuous_update=False, readout_format='.4f'),
          LQ1 = widgets.FloatSlider(min=0.49305795,max=3,step=0.16434205,value=0.6574, description='Ld Q1 (GVAr)',
                                    continuous_update=False, readout_format='.4f'),
          LP2 = widgets.FloatSlider(min=0.8,max=5,step=0.4,value=1.6, description='Ld P2 (GW)',
                                    continuous_update=False, readout_format='.4f'),
          LQ2 = widgets.FloatSlider(min=0.39442636,max=2,step=0.13147364,value=0.5259, description='Ld Q2 (GVAr)',
                                    continuous_update=False, readout_format='.4f'),
          LP3 = widgets.FloatSlider(min=4,max=40,step=4,value=16, description='Ld P3 (GW)',
                                    continuous_update=False, readout_format='.4f'),
          LQ3 = widgets.FloatSlider(min=3.94416358,max=20,step=1.31473642,value=5.2589, description='Ld Q3 (GVAr)',
                                    continuous_update=False, readout_format='.4f'),
          GP1 = widgets.FloatSlider(min=1,max=10,step=0.5,value=3, description='G P1 (GW)',
                                    continuous_update=False, readout_format='.4f'),
          Wind = widgets.FloatSlider(min=0,max=6,step=0.1,value=3, description='Wind (GW)',
                                     continuous_update=False, readout_format='.4f')):

    # Get active and reactive loads
    Base = 1/10
    P = [LP1/Base, LP2/Base, LP3/Base]
    Q = [LQ1/Base, LQ2/Base, LQ3/Base]

    Connectivity = [
        [1, 2, complex(0.0003, 0.0030), 1.2],
        [2, 3, complex(0.0015, 0.0150), 2.2]
    ]
    Load = [
        [1, complex(P[0], Q[0])],
        [2, complex(P[1], Q[1])],
        [3, complex(P[2], Q[2])]
    ]
    Generator = [
        {'Bus':3, 'V':1, '𝜃':0 },
        {'Bus':1, 'P':(GP1+Wind)/Base, 'V':1.02 }
    ]

    Ybus = get_Ybus(Connectivity, True, False)

    P_Data, Q_Data = develop_PF_Equations(Load, Generator, Ybus, True, False)
    Bus_Data, Bus_Type = get_Bus_Type(Ybus, Load, Generator)

    V_All, 𝜃_All, Threshold, Succes = Newtons_Method(P_Data, Q_Data, Bus_Data, Bus_Type, Generator, 0)
    Results = Visualize_Lab(Connectivity, Load, P, Q, Base, V_All, 𝜃_All, Succes)

    print('\nG1 above is the combined output of both generators:')
    print('\tThe Wind generator in Scotland is producing %.4f GW'%Wind)
    G = Results['Net_Power'][0]/1000 + Load[0][1]*Base
    print('\tThe synchronous generator in Scotland is producing %.4f + j %.4f GVA'%(GP1, G.imag))

interactive(children=(FloatSlider(value=2.0, continuous_update=False, description='Ld P1 (GW)', max=5.0, min=1…

[Back to top](#EEEN30131-Power-Flow-Analysis-&-Power-System-Control-Coursework)

#### A.2.2 (Report) [3 marks]

Using the tool presented above in Section A.2.1, find the maximum power output of the non-synchronous generator in Scotland (to the nearest 100 MW) that can be accommodated by the system before the system voltage limits are violated.

Make sure to clearly present this value in your report, and discuss why the system cannot withstand higher power outputs from the non-synchronous generator in Scotland.

[Back to top](#EEEN30131-Power-Flow-Analysis-&-Power-System-Control-Coursework)

#### A.2.3 (Report) [10 marks]

Discuss the results you have obtained. [10 marks]
You might want to include some of the following in your discussion:
- Descriptions and explanations about how the system parameters change with varying loading.
- Make sure you consider the different parameters in the test system that vary.
- Data visualisation that supports your discussion.


[Back to top](#EEEN30131-Power-Flow-Analysis-&-Power-System-Control-Coursework)

## Part B - Power System Control

In this part of the laboratory, you will explore different control actions that can be taken to keep the power system operating within statutory limits, as well as to facilitate the integration of larger volumes of non-synchronous generation.

To complete this part of the laboratory, you will have to complete three activities worth 17, 13 and 20 marks, respectively:
- Task B.1 involves simulating the primary response of two interconnected areas as synchronous generators are replaced with low carbon technologies and inertia decreases.
- Task B.2 involves analysing the frequency response of generators and loads in a single area.
- Task B.3 involves a new series of power flow studies with the simplified GB system.

[Back to top](#EEEN30131-Power-Flow-Analysis-&-Power-System-Control-Coursework)

### Task B.1 - Primary frequency control for systems with low inertia

This task involves two activities worth 10 and 7 marks, respectively:
- B.1.1: Use the tool presented to simulate the primary frequency response of the interconnected areas depicted in Figure 2.
- B.1.2: Analyse your results and discuss your findings in the report.

![Week06_Interconnected.png](Figures/Week06_Interconnected.png)
<div align="center">
Figure 2: Two interconnected areas.
</div>

[Back to top](#EEEN30131-Power-Flow-Analysis-&-Power-System-Control-Coursework)

#### B.1.1 (Record in excel template) [10 marks]

To complete this activity you will have to:
- Consider the following model settings:
  - $P_{L}^{ref}$: The reference power settings are zero in both areas.
  - $K_{G}$: The integral gain is 100 pu in area 1 and 53.333 pu in area 2.
  - $T_{CH}$: The charging time parameter is 0.5 pu in area 1 and 0.6 pu in area 2.
  - $R$: The regulation constant (droop characteristic) is 0.05 pu in area 1 and 0.0625 pu in area 2.
  - $M$: The angular momentum is 20 pu in area 1 and 10 pu in area 2.
  - $D$: The load damping is 0.6 pu in area 1 and 0.9 pu in area 2.
  - A generator in area 1 with a capacity of 2000 MW and a generator in area 2 with a capacity of 1000 MW.
  - A contingency when the demand is suddenly 300 MW greater than demand.
  - A 1000 MVA, 50 Hz base.
- Gradually reduce the inertia in area 1 by decreasing the angular momentum by one pu and the capacity of the generator by 10 MW.
- If the frequency response of the system is stable after 30 seconds , record the nadir (lowest system frequency) and system settling frequency (new equilibrium) for each simulation.

- If the frequency response of the system is unstable after 30 seconds, stop the study and do not attempt to record the nadir or settling frequency.

In [9]:
@interact
def A_1_1(Area = widgets.FloatSlider(min=1,max=2,step=1,value=1, description='Area',
                                    continuous_update=False, readout_format='.0f'),
          Contingency = widgets.FloatSlider(min=-1000,max=1000,step=100,value=-300, 
                                            description='Fault (MW)',
                                            continuous_update=False, readout_format='.0f'),
          M1 = widgets.FloatSlider(min=0,max=20,step=1,value=20, description='M1 (kgm2/s)',
                                   continuous_update=False, readout_format='.0f'),
          M2 = widgets.FloatSlider(min=0,max=20,step=1,value=10, description='M2 (kgm2/s)',
                                   continuous_update=False, readout_format='.0f'),
          Cap1 = widgets.FloatSlider(min=1700,max=2000,step=10,value=2000, description='Cap.1 (MW)',
                                     continuous_update=False, readout_format='.0f'),
          Cap2 = widgets.FloatSlider(min=100,max=2000,step=100,value=1000, description='Cap.2 (MW)',
                                     continuous_update=False, readout_format='.0f')):
    Dist = {
        'Area': Area,
        'Magnitude': Contingency,  # Positive: Generation is higher than load
        'Flow': 0  # From area 1 to 2 by default
    }
    Gen = [
        {'Area':1, 'Capacity': Cap1, 'R':10, 'Units': '%', 'Output': 500},
        {'Area':2, 'Capacity': Cap2, 'R':4.5, 'Units': '%', 'Output': 500}
    ]
    Loads = [
        {'D':0.6, 'Units': 'pu'},
        {'D':0.9, 'Units': 'pu'}
    ]
    BaseD = 1000
    F = 50
    T = 30
    
    # System data
    Tch = [0.5, 0.6]
    Kg = [100, 53.3333]
    Tl = 2

    Disturbance, Generators, Damping = get_pu(Dist, Gen, Loads, BaseD, F)
    Δw, _, _ = get_Δw(Disturbance, Generators, Damping, F)

    # Tie line
    R, D = get_Tie_Data(Dist, Generators, Damping, Kg)
    Tg = [1/Kg[0]/R[0], 1/Kg[1]/R[1]]

    # Create input signals
    t = numpy.linspace(0,T,1000)
    ΔPL = get_step(t, 1, Disturbance['Magnitude'])
    if M1==0:
        M1=0.1
    M = [M1, M2]

    # Model frequency response
    Δw1, Δw2, mod = \
        interconnected_frequency_primary(Dist['Area'], R, D, M, Tch,
                                         Kg, Tg, Tl, t, ΔPL)

    # Get nadir and settling
    aux = max(Δw1[900:])-min(Δw1[900:])+max(Δw2[900:])-min(Δw2[900:])
    Nadir = (1+min([min(Δw1), min(Δw2)]))
    print('A %6.2f pu (%.0f MW) contingency is simulated in area %d\n'
          %(Contingency/BaseD, Contingency, Area))

    plot_Δw(t, Δw1, Δw2)

    if aux < 0.0005:
        Settling = (1+Δw1[-1])
        print('The nadir is %10.4f pu (%10.4f Hz))'%(Nadir, Nadir*F))
        print('The settling is %10.4f pu (%10.4f Hz))'%(Settling, Settling*F))
    else:
        print('The system is now unstable after %d seconds'%T)
    

interactive(children=(FloatSlider(value=1.0, continuous_update=False, description='Area', max=2.0, min=1.0, re…

[Back to top](#EEEN30131-Power-Flow-Analysis-&-Power-System-Control-Coursework)

#### B.1.2 (Report) [7 marks]

Discuss the results you have obtained

You might want to include some of the following in your discussion:
- Description of the study.
- The lowest angular momentum in area 1 (M1) that allows the system to reach a stable frequency equilibrium after 30 seconds.
- Descriptions and explanations about how the frequency stability metrics (nadir and settling frequency) change as the angular momentum varies.
- Data visualisation that supports your discussion. 

[Back to top](#EEEN30131-Power-Flow-Analysis-&-Power-System-Control-Coursework)

### Task B.2 - Primary frequency control – Contributions of loads

This task involves two activities worth 10 and 5 marks, respectively:
- B.2.1: Calculate the primary frequency response of the generators and loads.
- B.2.2: Analyse your results and discuss your findings in the report.

[Back to top](#EEEN30131-Power-Flow-Analysis-&-Power-System-Control-Coursework)

#### B.2.1 (Record in excel template) [10 marks]

Use the information below to calculate the primary frequency response of the synchronous generators and loads:
- An area faces a contingency where demand is suddenly greater than generation by 300 MW.

- Two generators are connected to the area, their regulation constants are $R_1=0.05 pu$ and $R_2=0.025 pu$.

- Two large loads are connected to the area, their damping constants are $D_1=5 pu$ and $D_2=10 pu$.

- The frequency drop caused by the contingency is $∆ω=-0.004 pu$ (-0.2 Hz). Accordingly, the new frequency of the system is 49.8 Hz.

- Consider a 1000 MW, 50 Hz base.

[Back to top](#EEEN30131-Power-Flow-Analysis-&-Power-System-Control-Coursework)

#### B.2.2 (Report) [5 marks]

Discuss the results you have obtained.

You might want to address the following questions in your discussion:
- What is the combined response of the synchronous generators and demands?
- How would the frequency response of the loads vary with different damping constraints?

[Back to top](#EEEN30131-Power-Flow-Analysis-&-Power-System-Control-Coursework)

### Task B.3 - Voltage control

This task involves two activities worth 10 and 5 marks, respectively:
- B.2.1: Use the tool provided to obtain voltage magnitudes and angles as well as active and reactive power flows across the lines of the power system shown in Figure 3.
- B.2.2: Analyse your results and discuss your findings in the report.

![Week06_CWK_Condenser.png](Figures/Week06_CWK_Condenser.png)
<div align="center">
Figure 3: Simplified GB system with added synchronous condenser.
</div>

#### B.3.1 (Record in excel template) [10 marks]

Set the output of the synchronous condenser to -250 MVAr, and simulate the system as the output of the condenser increases to 250 MVAr in steps of 50 MVAr.

In [10]:
@interact
def A_1_1(LP1 = widgets.FloatSlider(min=1,max=5,step=0.5,value=2, description='Ld P1 (GW)',
                                    continuous_update=False, readout_format='.4f'),
          LQ1 = widgets.FloatSlider(min=0.49305795,max=3,step=0.16434205,value=0.6574, description='Ld Q1 (GVAr)',
                                    continuous_update=False, readout_format='.4f'),
          LP2 = widgets.FloatSlider(min=0.8,max=5,step=0.4,value=1.6, description='Ld P2 (GW)',
                                    continuous_update=False, readout_format='.4f'),
          LQ2 = widgets.FloatSlider(min=0.39442636,max=2,step=0.13147364,value=0.5259, description='Ld Q2 (GVAr)',
                                    continuous_update=False, readout_format='.4f'),
          LP3 = widgets.FloatSlider(min=4,max=40,step=4,value=16, description='Ld P3 (GW)',
                                    continuous_update=False, readout_format='.4f'),
          LQ3 = widgets.FloatSlider(min=3.94416358,max=20,step=1.31473642,value=5.2589, description='Ld Q3 (GVAr)',
                                    continuous_update=False, readout_format='.4f'),
          SC = widgets.FloatSlider(min=-300,max=300,step=50,value=-50, description='SC (MVAr)',
                                    continuous_update=False, readout_format='.4f')):

    # Get active and reactive loads
    Base = 1/10
    P = [LP1/Base, LP2/Base, LP3/Base]
    Q = [LQ1/Base, LQ2/Base, LQ3/Base]

    Connectivity = [
        [1, 2, complex(0.0003, 0.0030), 1.2],
        [2, 3, complex(0.0015, 0.0150), 2.2]
    ]
    Load = [
        [1, complex(P[0], Q[0])],
        [2, complex(P[1], Q[1])],
        [3, complex(P[2], Q[2])]
    ]
    Generator = [
        {'Bus':1, 'V':1, '𝜃':0 },
        {'Bus':3, 'P':P[2], 'Q':Q[2]+SC/100}
    ]

    Ybus = get_Ybus(Connectivity, True, False)

    P_Data, Q_Data = develop_PF_Equations(Load, Generator, Ybus, True, False)
    Bus_Data, Bus_Type = get_Bus_Type(Ybus, Load, Generator)

    V_All, 𝜃_All, Threshold, Succes = Newtons_Method(P_Data, Q_Data, Bus_Data, Bus_Type, Generator, 0)
    Results = Visualize_Lab(Connectivity, Load, P, Q, Base, V_All, 𝜃_All, Succes)

    print('\nG3 above is the combined output of the generator and condenser:')
    #print('\tThe Wind generator in Scotland is producing %.4f GW'%Wind)
    #G = Results['Net_Power'][0]/1000 + Load[0][1]*Base
    #print('\tThe synchronous generator in Scotland is producing %.4f + j %.4f GVA'%(GP1, G.imag))

interactive(children=(FloatSlider(value=2.0, continuous_update=False, description='Ld P1 (GW)', max=5.0, min=1…

[Back to top](#EEEN30131-Power-Flow-Analysis-&-Power-System-Control-Coursework)

#### B.3.2 (Report) [5 marks]

Discuss the results you have obtained.

You might want to include some of the following in your discussion:
- Description of the study.
- Visualization of the results.
- The impacts of reactive injections at the receiving end of a line (Bus 3).


[Back to top](#EEEN30131-Power-Flow-Analysis-&-Power-System-Control-Coursework)

## Summary

Your excel template should include results for:

| Task/Discussion   | Description                                                                      | Marks |
| ----------------- | -------------------------------------------------------------------------------- | ------|
| Task A.1.1        |system parameters for different loading conditions at power factor of 0.95 lagging| 10    |
| Task A.1.2        |system parameters for different loading conditions at power factor of 0.92 lagging| 10    |
| Task B.1.1        |system nadir and settling frequencies                                             | 10    |
| Task B.2.1        |calculated generation and load values                                             | 10    |
| Task B.3.1        |system parameters under different reactive power injections                       | 10    |

Your report should include:

| Task/Discussion   | Description                                                                      | Marks |
| ----------------- | -------------------------------------------------------------------------------- | ------|
| Discussion A.1.3  |about the results obtained for tasks A.1.1 and A.1.2, the power flow simulations at different power factors| 10    |
| Task A.2.1        |the system parameters as the non-synchronous generation in Scotland increases its output (until unacceptable voltages are seen)| 7     |
| Task A.2.2        |statement on the maximum allowable value of Scottish non-synchronous generation                                             | 3     |
| Discussion A.2.3  |about the results obtained in tasks A.2.1 and A.2.2                                             | 10    |
| Discussion B.1.3  |about the results obtained for tasks B.1.1 and B.1.2, frequency response results for different levels of inertia                                             | 10    |
| Discussion B.2.2  |about the results obtained in task B.2.1, the generation and load variation values based on your calculations                                             | 5     |
| Discussion B.3.3  |about the results obtained in task B.3.1, the voltages and flows subject to different reactive power injections                                             | 5     |

[Back to top](#EEEN30131-Power-Flow-Analysis-&-Power-System-Control-Coursework)